In [10]:
# 3/2017 | Moss McLaughlin

In [11]:
import sys
import csv
import itertools
import time
import operator
import io
import array
from datetime import datetime

import numpy as np
import nltk

In [12]:
SENTENCE_START_TOKEN = "sentence_start"
SENTENCE_END_TOKEN = "sentence_end"
UNKNOWN_TOKEN = "<UNK>"
ARTICLE_END_TOKEN = "</ARTICLE_END>"
NUM_TOKEN = "<NUM>"

similarity_threshold = 0.8

## Load Pre-trained GloVe emeddings

Load pre-trained GloVe word emeddings and build a dictionary.
Dataset has vocabulary of 400,000.

Example lines (500,501,502) of glove.6B.50d.txt (words are 50 dimensional vectors):

college -1.2283 1.4176 -0.68625 -1.1615 -0.0019627 -0.52577 -1.5977 -0.25307 -0.21699 -0.56572 -0.14248 0.15765 0.18763 -1.3173 -0.53438 0.20573 -0.47985 1.0476 -0.5075 0.77326 0.39774 0.70346 -0.32232 0.60171 -0.13352 -1.842 -0.092598 -1.1288 -1.4859 -0.86235 2.5976 0.66146 -0.054094 -1.5755 1.1185 0.13911 -0.26915 0.57159 1.3328 -0.24861 -0.68554 -0.43956 -0.86744 0.95772 -0.67349 0.36049 0.61952 -0.26356 -0.3715 0.32093

working 0.25792 -0.14413 -0.035634 -0.60551 0.11004 -0.058799 -1.2209 -0.031605 -0.023699 -0.37419 0.28924 0.12331 -0.31903 0.65017 0.28362 -0.20956 0.30423 0.75571 0.47964 -0.41976 0.68923 0.92026 0.070798 0.3948 0.24721 -1.4038 -0.14209 -0.6946 -0.035052 0.0041205 3.4024 0.036271 -0.58483 -0.72107 0.036996 0.33065 -0.27332 0.51897 0.3499 0.061199 -0.36178 -0.26534 0.4271 0.0081181 0.19844 -0.38564 -0.35535 0.032932 -0.50055 0.54358

community 0.14774 0.62713 -0.81852 -0.16878 0.44055 -0.12515 -0.87369 -0.060084 0.48804 -0.23463 -0.017574 -0.70673 
0.77392 -0.82772 -0.69719 0.076588 0.98306 0.58452 0.77501 0.30231 0.0059052 0.58345 -1.0178 1.1176 -0.13487 -1.2102 -0.095842 -0.70611 -0.54565 -0.12818 3.4346 0.12549 -0.35702 -1.3685 -0.32705 -0.25489 -0.52943 -0.90213 0.28179 0.22691 -0.47532 -0.51934 -0.32232 0.07629 -0.10132 0.47723 -0.83698 0.021588 -0.36972 0.14843

The dataset is ordered from most to least frequent tokens.
The three most frequent tokens are ['the'], [','], ['.'].  

We can build a smaller dictionary to increase speed by only reading the first n lines of glove.6B.50d.txt   

In [13]:
def load_embeddings():
  print("Loading word embeddings...")
  with open('../word_embedding/glove.6B/glove.6B.50d.txt') as f:
    words = {}
    w = [line.split(' ') for line in f]
    #print(w[0])
    v = [x[1:199999] for x in w] # here is where you can edit embedding vocab size 
    w = [x[0] for x in w]
    print("Word embedding vocab size: ",len(v)-1,'\n')
    

    for i in range(len(w)):
      words[w[i]] = v[i]

    return(words)


In [14]:
def create_embedding(vocab_size,itw):
    print("Building word embedding matrix...")
    E = [None] * vocab_size
    embedding_dict = load_embeddings()
    for i in range(vocab_size): E[i] = embedding_dict[itw[i]]
    return E
        

In [15]:
def cos_similarity(x,y):
    x = np.array(x).astype(np.float)
    y = np.array(y).astype(np.float)
    
    
    d = np.dot(x,y) / (np.sqrt(np.dot(x,x))*(np.sqrt(np.dot(y,y))))
    return d

In [16]:
def find_similar_words(word,word_list,embeddings,similarity_threshold):
    similarity = [cos_similarity(embeddings[word],embeddings[w[0]]) for w in word_list]
    similarity = np.array(similarity)
    if similarity[similarity.argmax()] > similarity_threshold:
        return word,similarity.argmax()
    

## Load and format training data

Function may vary based on training data file format.

We format and tokenize our training data, then make a list of the most frequent words. Here we choose the vocab size our model will know. 

In [17]:
def load_data(filename="../score/data.txt", vocabulary_size=2000, min_sent_characters=10):
    word_to_index = []
    index_to_word = []
    print("Reading text file...")
    with open(filename, 'rt') as f:
        data = [line.split('|') for line in f.readlines()]
        print("Raw training data: ",data[0][0])
        #txt = [sent[0] for sent in data]
        txt = ["%s 'sentence_end'" % sent[0] for sent in data]
        
        
    print("Parsed %d articles." % (len(txt)))
    tokenized_sentences = [nltk.word_tokenize(line.lower()) for line in txt]
    
    # Filter Words.
    print("Filtering words...")
    tokenized_sentences = [[w[1:] for w in line] for line in tokenized_sentences]
    tokenized_sentences = [[w for w in line if not w==''] for line in tokenized_sentences]
    tokenized_sentences = [[w for w in line if '\\' not in w] for line in tokenized_sentences]
    tokenized_sentences = [[w for w in line if '*' not in w] for line in tokenized_sentences]
    tokenized_sentences = [[w for w in line if '[' not in w] for line in tokenized_sentences]
    tokenized_sentences = [[w for w in line if ']' not in w] for line in tokenized_sentences]
    tokenized_sentences = [[w for w in line if '"' not in w] for line in tokenized_sentences]
    tokenized_sentences = [[w for w in line if "'" not in w] for line in tokenized_sentences]
    tokenized_sentences = [[w for w in line if "`" not in w] for line in tokenized_sentences]
    
    # Replace all numbers with num_token
    for i,line in enumerate(tokenized_sentences):
        tokenized_sentences[i] = [w if w.isnumeric()==False else NUM_TOKEN for w in line]
        
    # Count word frequencies and build vocab
    word_freq = nltk.FreqDist(itertools.chain(*tokenized_sentences))
    n_data_words = len(word_freq.items())
    print("Found %d unique words tokens.\n" % n_data_words)

    
    embeddings = load_embeddings()
    vocab = sorted(word_freq.items(), key=lambda x: (x[1], x[0]), reverse=True)
    vocab = [w for w in vocab if w[0] in embeddings]
    
    print("Found %d out of %d words in GloVe embeddings." % (len(vocab),n_data_words))
    
    # We take the [vocabulary_size] most frequent words and build our word embedding matrix (or lookup table for now).  
    # Words in dataset are now either inside or outside embedding matrix.
    inside_words = sorted(vocab[:vocabulary_size], key=operator.itemgetter(1))
    outside_words = sorted(vocab[vocabulary_size:], key=operator.itemgetter(1))
    
    index_to_word = ["<MASK/>", UNKNOWN_TOKEN,SENTENCE_END_TOKEN] + [x[0] for x in inside_words]
    word_to_index = dict([(w, i) for i, w in enumerate(index_to_word)])
    
    print("Using vocabulary size %d." % vocabulary_size)
    print("The least frequent word in our vocablary is '%s' and appeared %d times in this dataset." \
          % (vocab[-1][0], vocab[-1][1]))
    
    # Find similar words that are in the data set but outside of our vocabulary
    similar_words = {}
    for w in outside_words:
        try: 
            similar_word,similar_index = find_similar_words(w[0],inside_words,embeddings,similarity_threshold)
            print("Replacing %s with %s" % (similar_word,inside_words[similar_index][0]))
            similar_words[similar_word] = inside_words[similar_index]
        except: None
        for line in tokenized_sentences:
            for x in line:
                if x in similar_words: x = similar_words[x] 
                    
    print('\nLeast freq words in vocab: ')
    print(inside_words[:25])
    print('\n')
    
    # Save word_to_index & index_to_word if model is to be loaded after training
    np.savez('index',itw=index_to_word,wti=word_to_index)
    
    # Filter sentences 
    for i,line in enumerate(tokenized_sentences):
        tokenized_sentences[i] = [w if w in word_to_index else UNKNOWN_TOKEN for w in line]
    tokenized_sentences = [s for s in tokenized_sentences if (len(s) > 1)]
    

    print('Filtered training data:')
    print(tokenized_sentences[0])
    print('\n')

    # Build training data
    X_train = np.asarray([[word_to_index[w] for w in sent[:-1]] for sent in tokenized_sentences])
    y_train = [line[1] for line in data]
    y_train = [10000*float(s.replace('\n','')) for s in y_train]
    return X_train,y_train,word_to_index,index_to_word
    

In [18]:
d = load_data('../score/data.txt',2000)

Reading text file...
Raw training data:  ['polypeptide', 'multiple', 'mainly', 'removal', 'cardinality', 'guarantee', 'indefinite', 'establishment', 'or', 'against', 'synchronized', 'recognition', 'algorithms', 'to', 'allow', 'way', 'latency', 'mentioned', 'reported']
Parsed 12200 articles.
Filtering words...
Found 5938 unique words tokens.

Loading word embeddings...
Word embedding vocab size:  400000 

Found 5710 out of 5938 words in GloVe embeddings.
Using vocabulary size 2000.
The least frequent word in our vocablary is 'acoustic' and appeared 1 times in this dataset.
Replacing biomedical with engineering
Replacing bin with al
Replacing season with game
Replacing reporting with report
Replacing profits with gains
Replacing polymerase with rna
Replacing pain with heart
Replacing ownership with property
Replacing movie with show
Replacing malaria with disease
Replacing incident with occurred
Replacing genus with species
Replacing gas with energy
Replacing bacterium with bacteria
Repl